In [195]:
from collections import defaultdict 
from sklearn.cluster import KMeans
import numpy as np
import random
import copy

class Node():

    def __init__(self, id, energy, location):
        self.id = id
        self.energy = energy
        self.location = location
        # List of lists, each route will look like ex. below
        self.routing_tbl = [[]] * 4
        self.hubID = -1

    def setTransmitPwr(self, transmit_pwr):
        self.transmit_pwr = transmit_pwr

    def setProcPower(self, processing_pwr):
        self.processing_pwr = processing_pwr

    def setHub(self, hubID):
        self.hubID = hubID

    # route entry will look like this: [seq.#, # of hops / energy it'll take, path]
    # # of hops / energy it'll take can def change to some other measurement of path cost
    # path will be a list so like 0 > 1 > 2 would be [0,1,2]
    def addRoute(self, var, index):
        # self.routing_tbl.append(route)
        if (index == 0):
            # THIS IS CHANGING ROUTING NUMBER
            self.routing_tbl[0] = 1
        elif (index == 1):
            self.routing_tbl[1] = var
        elif (index == 2):
            self.routing_tbl[2] = var
        elif (index == 3):
            self.routing_tbl[3] = var

class Graph(): 

    def __init__(self, vertices): 
        self.V = vertices
        self.graph = defaultdict(list)

    def addEdge(self, u, v):
        self.graph[u].append(v)

    def getAllPathsUtil(self, u, d, visited, path):

        visited[u] = True
        path.append(u)
        
        # If node equals destination
        if u == d:
            allPossiblePaths.append(copy.copy(path))
        else:
            for i in self.graph[u]:
                if visited[i] == False:
                    self.getAllPathsUtil(i, d, visited, path)

        path.pop()
        visited[u] = False

    def getAllPaths(self, s, d):
        visited = [False] * (self.V)

        path = []

        self.getAllPathsUtil(s, d, visited, path)

# Function calculates distance between two points
def dist(p1, p2, d): 
      
    x = []
    for i in range(d):
        x.append(p1[i] - p2[i])
        
    tot = 0
    for i in range(d):
        tot = tot + (x[i] * x[i])
        
    return tot

# takes in nodes, dest, and k (num of clusters); returns network 'layout'
# can use this function to update/change the layout as energy depletes
# or transmission power changes greatly because one of the nodes died/ran out of energy
def layout(nodes, dest, k):

    node_locs = np.zeros(shape=(len(nodes)+1, 2))
    for n in range(len(nodes)):
        node_locs[n] = nodes[n].location
    node_locs[len(nodes)] = dest
    kmeans = KMeans(n_clusters=k).fit(node_locs)
    clusters = []
    for i in range(k):
        clusters.append([])
    for l in range(len(kmeans.labels_)):
        for i in range(k):
            if kmeans.labels_[l] == i:
                clusters[i].append(node_locs[l])
    
    for c in clusters:
        h = c[0] # initialize h with possible hub for cluster c
        proximity = 100
        for i in range(len(c)):
            temp = 0
            for j in range(len(c)):
                temp += dist(c[i], c[j], 2)
            for n in nodes:
                if np.all(n.location == c[i]):
                    n.setTransmitPwr(float(temp) / float(len(nodes)))
            if temp < proximity:
                proximity = temp
                for n in nodes:
                    if np.all(n.location == c[i]):
                        h = n
        for l in c:
            for n in nodes:
                if np.all(n.location == l):
                    n.setHub(h.id)

# TO-DO: we need to add something in the sendPacket function or maybe outside when we call it
# to check for some threshold being hit to where we'd wanna change the layout. I'm thinking the
# threshold would be one of the nodes having low energy (let's say energy=2) so we'd call layout
# to a) change it from hub to one-hop if it's a hub to start or b) perhaps create another hub, one
# much closer to the dying node so that it can continue to send packets at a lower transmission
# power

# 'runs' the network / starts sending packets
def sendPacket(nodes, src, dest):

    packets = 0 # var for how many packets were sent before failure
    packet_size = 512 # let's say all packets being sent are of size 512 B
    
    hubs = [nodes[nodes[0].hubID]]
    for n in nodes:
        if nodes[n.hubID] not in hubs:
            hubs.append(nodes[n.hubID])

    # there is only one hub to start so it's the only one that can go
    # to destination, all the other nodes can only send info to hub
    if len(hubs) == 1:
        hub = hubs[0]
        hub.energy -= (packet_size * (hub.transmit_pwr / 1000))
        if src != hub.id:
            # update energy left after packet is sent
            nodes[src].energy -= (packet_size * (nodes[src].transmit_pwr / 1000))
            # make sure there was enough energy to send it
            if nodes[src].energy >= 0 and hub.energy >= 0:
                packets +=1 # energy left >= 0 so packet was sent successfully
        else:
            if hub.energy >= 0:
                packets += 1
    else:
        # num of vertices = num of hubs + 1 (+1 to include destination node)
        g = Graph(len(hubs) + 1)
        for h in hubs:
            for h2 in hubs:
                if h != h2:
                    g.addEdge(h.id, h2.id)
            g.addEdge(h.id, -1)
                
        global allPossiblePaths 
        allPossiblePaths = []
        
        # If src equals the hub
        if src == nodes[src].hubID:
            g.getAllPaths(src, -1)
        else:
            g.getAllPaths(nodes[src].hubID, -1)
        
        
        optimalPath = []
        allPathCosts = []
        
        print ("ALL POSSIBLE ", allPossiblePaths)
        for path in allPossiblePaths:
            path_cost = 0
            for p in path:
                if p != -1:
                    # Implement processesing power later
                    path_cost += (packet_size * (nodes[p].transmit_pwr / 1000))
            allPathCosts.append(path_cost)
            print("PATH: " + str(path) + '\n' + "PATH COST: " + str(path_cost))
        
        allPathCosts = np.array(allPathCosts)
        # Best path is lowest index
        bestPathIndex = np.argmin(allPathCosts)
        bestPath = allPossiblePaths[bestPathIndex]
       
        # Optimal path from a hub to destination, when multiple hubs
        # Should loop for all hubs?
        print ("BEST PATH ", bestPath)
        print ("NODES SRC HUB ID IS ", nodes[src].hubID)
        optimalPathOfHubs[nodes[src].hubID] = bestPath
        
        # WE CAN ALSO DO THE SET ROUTE FROM NODE CLASS
        nodes[nodes[src].hubID].addRoute(bestPath, 3)
        
        # Now getting number of hops
        hops = 0
        for i in range(0, len(bestPath)):
            # CHANGE IF WE DONT COUNT DESTINATION AS A HOP
            if (nodes[src].hubID != bestPath[i]):
                hops += 1
        
        # Adding the number of hops to the table
        nodes[nodes[src].hubID].addRoute(hops, 1)
        # Also adding the cost of the route to the tabl
        nodes[nodes[src].hubID].addRoute(allPathCosts[bestPathIndex], 2)
        
        print ("ROUTING TABLE SO FAR ", nodes[nodes[src].hubID].routing_tbl)
        
        # Then subtract from src node ONLY if it's not a hub
        # Checking if src is one of the hubs
        counter = 0
        
        for i in range(0, len(hubs)):
            if (nodes[src].id == hubs[i].id):
                counter += 1
        
        # If counter is 0, then it's not a hub
        # Else if counter is > 0 then it is a hub, and we'll subtract energy in the below function
        
        success = configureMultiNodePath(nodes, counter, src, bestPath)
        # Send best path and do all the energy changing, if success return 1, and if 1 then a packet has been sent
        if (success == True):
            packets += 1

    return packets

def configureMultiNodePath(nodes, counter, src, path):

    iteratedNodes = []
    packet_size = 512
    
    # If the src node is NOT a hub
    if counter == 0:
            nodes[src].energy -= (packet_size * (nodes[src].transmit_pwr / 1000))
            iteratedNodes.append(nodes[src])
            
    # If the src node is a hub
    else: 
        # Change the power in the nodes in the path, don't iterate over the last index (which is the destination)
        for i in range(0, len(path) - 1):
            nodes[i].energy -= (packet_size * (nodes[src].transmit_pwr / 1000))
            iteratedNodes.append(nodes[i])
            
    
    # Checking if all nodes that we changed were capable of sending the packet
    count = 0
    for i in range(0, len(iteratedNodes)):
        if (iteratedNodes[i].energy >= 0):
            count += 1
    
    # If count equals the len of the nodes on the path, then all of them have enough energy
    if (count == len(iteratedNodes)):
        return True
    else:
        return False
    
# Can use this to check if all nodes are out of energy, implemented in commented stuff below
def checkNodeEnergy(nodes):
    counter = 0
    for i in range(0, len(nodes)):
        if (nodes[i].energy >= 0):
            counter += 1

    # Then all nodes have enough power
    if (counter > 0):
        return True
    # No nodes have good enough power
    elif (counter == 0):
        return False

    return False

def checkHubsAvailability(nodes):
    # Get all known hubs and check if they need to change to a 1 hop
    knownHubs = []
    for i in range (0, len(nodes)):
        knownHubs.append(nodes[i].hubID)
    
    # Remove duplicates
    knownHubs = list(dict.fromkeys(knownHubs))
    # Get a list of the hubs we should change
    hubsNeededToBeChanged = []
    for i in range(0, len(knownHubs)):
        # If the hubs energy is lower than 0.5, safe to assume is ded, but can change the number
        if (nodes[knownHubs[i]].energy < 0.5):
            hubsNeededToBeChanged.append(nodes[knownHubs[i]].hubID)
    
    return hubsNeededToBeChanged
    
    
# This is just a simple idea, its probably more complex but i think idea is there :)    
def changeHubs(nodes,hubsToBeChanged):
    # Change to other viable node
    print ("HUBS TO BE CHANGED ", hubsToBeChanged)
    
    for i in range(0, len(hubsToBeChanged)):
        for j in range(0, len(nodes)):
            if (nodes[j].id != hubsToBeChanged[i]):
                # Matching hubIDs
                if (nodes[j].hubID == hubsToBeChanged[i]):
                    # Switch it to another node
                    if (nodes[j].energy < 0.5):
                        print ("THEY ALL DED LOL!")
                    else:
                        print ("Changed hub to", nodes[j].id)
                        nodes[j].hubID = nodes[j].id
                    
        
    
# destination / gateway location
dest = [5, 6]

# sensor nodes - initialized with node locations and they'll
# all start with same amount of energy
n0 = Node(0, 1, [3, 5])
n1 = Node(1, 1, [3, 1])
n2 = Node(2, 1, [1, 3])
n3 = Node(3, 1, [6, 3])

# list of nodes so it's easy to pass them to a function
nodes = [n0, n1, n2, n3]

# determine how many clusters want for the network to start
#k = int(len(nodes) / 4)
k = 2 # just doing this so i can see if k > 1 part of sendPackets is working
# create initial network layout
layout(nodes, dest, k)

# start sending packets
total_packets = 0

# TO-DO: put random generation of src node and calling of sendPacket into a loop
# that breaks once no more packets can be sent aka all the nodes have no energy left

# This will hold the best path to the destination 
# Index of this list is the node, and it will hold the optimal path to the dest
# THIS IS ANOTHER WAY TO VIEW THE ROUTING TABLE FROM THE NODE CLASS I THINK, MAYBE CAN REMOVE IF DONT WANT
optimalPathOfHubs = [[]] * len(nodes)



# randomly generated source node
s = random.randint(0, 3)
print("SRC IS NODE: " + str(s))
p = sendPacket(nodes, s, dest)
print ("OPTIMAL PATH OF HUBS", optimalPathOfHubs)

hubsToBeChanged = checkHubsAvailability(nodes)
if not hubsToBeChanged:
    print ("NO CHANGE HUB IS GOOD :)")
else:
    print ("NOPERS HUB IS BAD >:(")
    changeHubs(nodes, hubsToBeChanged)
    # CHANGE HUBS HERE


# Change hubs in layout() when hubs are down, will probably have this in a loop





# s = random.randint(0, 3)
# print ("INITIAL SOURCE: ", s)

# Ignore this for now
# if (nodes[s].id != one_hop):
    # Then it's a hub

# WHILE LOOP TO KEEP SENDING PACKETS
# !!!!!!!!!!!!!!!!!!!!!!!
# CAN USE THE STUFF DOWN HERE, BUT HAVE TO FIX SINCE WE DONT USE HUB ANYMORE
# HAVE TO FIX HUB TO BECAUSE WE CHANGE HUBS IN LAYOUT
# !!!!!!!!!!!!!!!!!!!!!!!!

# Keep track of one hops and hubs
# usedHubs  = []
# usedSources = []
# usedHubs.append(hub.id)
# Check if all nodes are usable
# continueFlag = True

# while (continueFlag == True):
    # Make sure source power for all nodes is good, if good then we can use it, else pick another node
    # if (checkNodeEnergy(nodes) == False):
        # continueFlag = False
        # print ("ALL SOURCES SUCK")
    # else:
        # 2 is our threshold, both the source and hub can send
        # if (nodes[s].energy > 2 and hub.energy > 2):
            # p = sendPacket(hub, one_hop, s, dest)
            # usedSources.append(s)
            # usedHubs.append(hub.id)
            # total_packets += 1
        # Change source node to something else, can also be a hub
        # else:
            # 
            # canUse = True
            # while (canUse == True):
                # if (s in usedSources and s in usedHubs):
                    # Cant use this one since it's already been used
                    # pass
                # s = random.randint(0,3)
                # hubInt = random.randint(0,3)
                # hub = nodes[hubInt]
                # canUse = False

        # print ("ALL USED :" , usedSources)
        # If 0 is not present in usedSources, it means it used up energy as a hub, will configure
        # this in a bit
            
# print ("PACKETS SENT: ", total_packets)
# for i in range(0, len(nodes)):
    # print (nodes[i].energy)

SRC IS NODE: 0
ALL POSSIBLE  [[0, 1, -1], [0, -1]]
PATH: [0, 1, -1]
PATH COST: 3.328
PATH: [0, -1]
PATH COST: 2.304
BEST PATH  [0, -1]
NODES SRC HUB ID IS  0
ROUTING TABLE SO FAR  [[], 1, 2.304, [0, -1]]
OPTIMAL PATH OF HUBS [[0, -1], [], [], []]
NOPERS HUB IS BAD >:(
HUBS TO BE CHANGED  [0]
Changed hub to 3
